<a href="https://colab.research.google.com/github/kmongsil1105/colab_ipynb/blob/main/AI_YOLOv5_my_objection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# yolov5 를 customize해서 나만의 object detection 모델을 만들어보기

1. yolov5 를 clone 

In [ ]:
 !git clone https://github.com/ultralytics/yolov5 

Cloning into 'yolov5'...
remote: Enumerating objects: 9339, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 9339 (delta 14), reused 20 (delta 7), pack-reused 9309
Receiving objects: 100% (9339/9339), 9.77 MiB | 28.57 MiB/s, done.
Resolving deltas: 100% (6485/6485), done.


In [ ]:
%pip install imgaug==0.2.5
%cd /content/yolov5   
!pip install -U -r requirements.txt  # install dependencies   


     |████████████████████████████████| 562 kB 43.1 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=127e2a66b5af955edcb2309f8b9042c3ec0b83d76f5402da1f9bdd5c1b0bc022
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0249261004990bccb0f27985c74ba26e49
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
/content/yolov5
     |████████████████████████████████| 10.3 MB 27.1 MB/s 
     |████████████████████████████████| 15.7 MB 182 kB/s 
     |████████████████████████████████| 49.9 MB 14 kB/s 
     |████████████████████████████████| 3.0 MB 37.3 MB/s 
     |████████████████████████████████| 636 kB 41.0 MB/s 
     |████████████████████████████████| 28.5 MB 50 kB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 292 kB 50.5 MB/s 
     |███████████████████████

2. customized 모델에 사용할 데이터셋 만들기

In [ ]:
! pip install -U PyYAML

In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
%cat /content/drive/MyDrive/dataset/data.yaml

train: /content/drive/MyDrive/images/train/  #train 데이터셋이 있는 디렉토리
val: /content/drive/MyDrive/images/val/     #val 데이터셋이 있는 디렉토리

# number of classes
nc : 2

# class names
name: ['car','man']

In [ ]:
#this is the model configuration we will use for our tutorial 

%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
#$ python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt
# python train.py --img 640 --batch 16 --epochs 5 --data data/coco128.yaml --cfg models/yolov5s.yaml --weights weights/yolov5s.pt
!python train.py --img 640 --batch 30 --epochs 10 --data /content/drive/MyDrive/dataset/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights /content/yolov5/yolov5/yolov5s.pt

train: weights=/content/yolov5/yolov5/yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/drive/MyDrive/dataset/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=30, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-433-g621b6d5 torch 1.9.0+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_